In [ ]:
import json, numpy as np, pandas as pd
import io
import re
import copy

In [ ]:
# Dictionary for category index

move_df = pd.read_csv(r"C:\Users\Cyril\Documents\project\mov_clean.csv")
item_df = pd.read_csv(r"C:\Users\Cyril\Documents\project\item_clean.csv")
ability_df = pd.read_csv(r"C:\Users\Cyril\Documents\project\ability_clean.csv")
species_df = pd.read_csv(r"C:\Users\Cyril\Documents\project\species_clean.csv")


In [ ]:
sp_status = {'NOR':0,'BRN' :1, 'YAR' :2, 'FRZ': 3, 'PAR': 4, 'PSN':5, 'SLP':6, 'TOX':7} # yawn at 2

In [ ]:
weather = {'NONE' :0,'HAIL':1, 'RAINDANCE' :2, 'SANDSTORM': 3, 'SUNNYDAY':4}

In [ ]:
terrian = {'_UNKNOWN':0, ' ELECTRIC TERRAIN': 1,' GRASSY TERRAIN':2,' MISTY TERRAIN':3, ' PSYCHIC TERRAIN':4}
extra_field = {'_UNKNOWN':0, ' GRAVITY':1,' HEAL BLOCK':2, ' MAGIC ROOM':3,  ' MUD SPORT':4, \
           ' MUD SPOT':5, ' WATER SPORT':7, ' WONDER ROOM':8}
TrickRoom = {'_UNKNOWN':0, ' TRICK ROOM':1}

In [ ]:
side_con =  { '_UNKNOWN':0, 'MOVE: AURORA VEIL':1, 'FIRE PLEDGE':2, 'G-MAX CANNONADE':3, \
             'G-MAX STEELSURGE':4, 'G-MAX VINE LASH':5, 'G-MAX VOLCALITH':6, \
             'G-MAX WILDFIRE':7, 'GRASS PLEDGE':8, 'MOVE: LIGHT SCREEN':9, \
             'MOVE: LUCKY CHANT':10, 'MIST':11, 'REFLECT':12, 'SAFEGUARD':13, 
             'SPIKES':14, 'MOVE: STEALTH ROCK':15, 'MOVE: STICKY WEB':16, 'MOVE: TAILWIND':17, 'MOVE: TOXIC SPIKES':18, 'WATER PLEDGE':19}

In [ ]:
stat_index ={'atk':16, 'def':17, 'spa':18, 'spd':19, 'spe':20, 'evasion':22, 'accuracy':22}

In [ ]:
move_ky = move_df.set_index('Move').T.to_dict('list')
item_ky = item_df.set_index('Item').T.to_dict('list')
ability_ky = ability_df.set_index('Ability').T.to_dict('list')
species_ky = species_df.set_index('Species').T.to_dict('list')


In [ ]:
def move_match(name):  # function to match move to index
    tmp = re.sub('[^A-Za-z0-9]+', '', name.lower())
    try:
        tmp = move_ky[tmp][0]
    except (RuntimeError, IndexError, KeyError):
        tmp = 999
    return tmp

In [ ]:
def item_match(name): # function to match item to index
    tmp = re.sub('[^A-Za-z0-9]+', '', name.lower())
    try:
        tmp = item_ky[tmp][0]
    except (RuntimeError, IndexError, KeyError):
        tmp = 999
    return tmp

In [ ]:
def ability_match(name): # function to match ability to index
    tmp = re.sub('[^A-Za-z0-9]+', '', name.lower())
    try:
        tmp = ability_ky[tmp][0]
    except (RuntimeError, IndexError, KeyError):
        tmp = 999
    return tmp

In [ ]:
def spec_match(name): # function to match species to index
    tmp = re.sub('[^A-Za-z0-9]+', '', name.lower())
    try:
        tmp = species_ky[tmp][0]
    except (RuntimeError, IndexError, KeyError):
        tmp = 999
    return tmp

In [ ]:
def pokemon_read(team):  # function to read pokemon team, return list (25)
    
    pm_list = [None]*(13+12)
    
    pm_list[0] = spec_match(team['species']) #hot-key pending
    pm_list[1] = item_match(team['item'])
    pm_list[2] = ability_match(team['ability'])
    
    
    pm_list[3] = (team['evs']['hp']/4 +team['ivs']['hp'])
    pm_list[4] = (team['evs']['atk']/4 +team['ivs']['atk'])
    pm_list[5] = (team['evs']['def']/4 +team['ivs']['def'])
    pm_list[6] = (team['evs']['spa']/4 +team['ivs']['spa'])
    pm_list[7] = (team['evs']['spd']/4 +team['ivs']['spd'])
    pm_list[8] = (team['evs']['spe']/4 +team['ivs']['spe'])
    
    pm_list[9] = move_match(team['moves'][0])
    try:
        pm_list[10] = move_match(team['moves'][1])
    except (RuntimeError, IndexError, NameError):
        pm_list[10] =(None)
    try:
        pm_list[11] = move_match(team['moves'][2])
    except (RuntimeError, IndexError, NameError):
        pm_list[11] = (None)
    try:
        pm_list[12] = move_match(team['moves'][3])
    except (RuntimeError, IndexError, NameError):
        pm_list[12] = (None)
    
    pm_list[13] = 1
    pm_list[14] = 0
    pm_list[15] = 0
    pm_list[16] = 0
    pm_list[17] = 0
    pm_list[18] = 0
    pm_list[19] = 0
    pm_list[20] = 0
    pm_list[21] = 0
    pm_list[21] = 1
    pm_list[22] = 0
    pm_list[23] = 1
    pm_list[24] = 0
    
    return pm_list

In [ ]:
    # explanation of pokemon list
    # 13 battle HP (% of current hitpoint)  
    # 14 status 
    # 15 status% (probability getting out of it in this turn)
    # 16 atk boost (-6 to 6)
    # 17 def boost (-6 to 6)
    # 18 spa boost (-6 to 6)
    # 19 spd boost (-6 to 6)
    # 20 spe boost (-6 to 6)
    # 21 Protect% (1 for able to do, drop after each protect)
    # 22 accuracy/evasion 
    # 23 Movable (1 moveable, 0 not-moveable)
    # 24 Dynmax (dynmax turn left)


In [ ]:
def read_teams(team1,team2):  # read both team, return pokemon(6,25)
    
    out1 = [None]*6
    out2 = [None]*6
    for i in range(0,6):
        try:
            out1[i]=pokemon_read(team1[i])
        except (RuntimeError, IndexError, NameError):
            print('error1')
            out1[i] = None
            
        try:
            out2[i]=pokemon_read(team2[i])
        except (RuntimeError, IndexError, NameError):
            print('error2')
            out2[i] = None
        
    return out1, out2



In [ ]:
def clearboost(line, player, pos1, pos2):  # function to clear boost for switch
    for i in range(5):
        line[player][pos1][16+i]=0
        line[player][pos2][16+i]=0

In [ ]:
def readaction(inputlog,k):  # read input action, return digitalize action outcome

    sp_adj = 0
    CountSw = 0
    out = [None]*2
    double_switch = 0
    
    try:
        tmp = inputlog[k].split()
        if tmp[1]=="move":
            out[0] =  move_match(tmp[2])
        elif tmp[1]=="switch":
            out[0] =  -int(re.sub('[^0-9]+', '',tmp[2]))
            CountSw +=1
        else:
            sp_adj=1
    except (RuntimeError, IndexError, ValueError):
        out[0] =None
    
    try:
        tmp = inputlog[k].split()
        if tmp[3-sp_adj]=="pass":
            out[1] =None
        elif tmp[3-sp_adj]=="move":
            out[1] =  move_match(tmp[4-sp_adj])
        elif tmp[3-sp_adj]=="switch":
            out[1] =  -int(re.sub('[^0-9]+', '',tmp[4-sp_adj]))
            CountSw +=1
        elif tmp[3-sp_adj].startswith("+")|tmp[3-sp_adj].startswith("dyn"):
            if tmp[4-sp_adj]=="move":
                out[1] =  move_match(tmp[5-sp_adj])
            elif tmp[4-sp_adj]=="switch":
                out[1] =  -int(re.sub('[^0-9]+', '',tmp[5-sp_adj]))
                CountSw +=1
            elif tmp[4-sp_adj].startswith("dyn"):
                if tmp[5-sp_adj]=="move":
                    out[1] =  move_match(tmp[6-sp_adj])
                elif tmp[5-sp_adj]=="switch":
                    out[1] =  -int(re.sub('[^0-9]+', '',tmp[6-sp_adj]))
                    CountSw +=1
    except (RuntimeError, IndexError, ValueError):
        out[1] =None

    double_switch = max(CountSw-1,0)
    
    return out, CountSw, double_switch

In [ ]:
def readbatlog(line, batlog, inputlog, k, switchNo, p2, DD): # read whole battle log for state features
    
    current = line
    upflag = 0
    CountSw = switchNo
    double_switch= 0
    
    if batlog.startswith("|turn|"):
        upflag = 1
        if line[4][0] > 0: # weather turn count
            line[4][1] += 1
        if line[5][0] > 0: # terrian turn count
            line[5][1] += 1
        if line[6][0] > 0: # extra_f turn count
            line[6][1] += 1
        if line[7][0] == 1: # trick turn count
            line[7][1] += 1
        for i in range(6):
            try:
                if line[2][i][14]==6: line[2][i][15]+=0.33
                if line[3][i][14]==6: line[2][i][15]+=0.33
            except (RuntimeError, IndexError, NameError, TypeError):
                None
            
        line[10], countSw1, _ =readaction(inputlog,k)
        line[11], countSw2, _ =readaction(inputlog,k+1)
        CountSw = countSw1+countSw2
        k = k+2
        
    elif batlog.startswith("|-boost|"):
        tmp = re.findall(r"[\w'“”(),&.’ -]+",(batlog)) # (boost,pos,Pok,stat,stage)
        if tmp[1]=="p1a": 
            line[2][line[0][1]-1][stat_index[tmp[3]]] =  int(tmp[4])
        elif tmp[1]=="p1b": 
            line[2][line[0][2]-1][stat_index[tmp[3]]] += int(tmp[4])
        elif tmp[1]=="p2a": 
            line[3][line[1][1]-1][stat_index[tmp[3]]] += int(tmp[4])
        else: 
            line[3][line[1][2]-1][stat_index[tmp[3]]] += int(tmp[4])
        
    elif batlog.startswith("|-unboost|"):
        tmp = re.findall(r"[\w'“”(),&.’ -]+",(batlog)) # (boost,pos,Pok,stat,stage)
        if tmp[1]=="p1a": 
            line[2][line[0][1]-1][stat_index[tmp[3]]] -= int(tmp[4])
        elif tmp[1]=="p1b": 
            line[2][line[0][2]-1][stat_index[tmp[3]]] -= int(tmp[4])
        elif tmp[1]=="p2a": 
            line[3][line[1][1]-1][stat_index[tmp[3]]] -= int(tmp[4])
        else: 
            line[3][line[1][2]-1][stat_index[tmp[3]]] -= int(tmp[4])
        
    elif batlog.startswith("|-damage|"):
        tmp = re.findall(r"[\w'“”(),&.’ -]+",(batlog)) # (dam,pos,Pok,hp-new/fiant,hp-old)
        
        if tmp[1]=="p1a":
            if tmp[3]=='0 fnt':
                line[2][line[0][1]-1][13] = 0.0
            else:
                line[2][line[0][1]-1][13] = float(tmp[3])/float(re.sub('[^0-9]+', '',tmp[4]))
        elif tmp[1]=="p1b":
            if tmp[3]=='0 fnt':
                line[2][line[0][2]-1][13] = 0.0
            else:
                line[2][line[0][2]-1][13] = float(tmp[3])/float(re.sub('[^0-9]+', '',tmp[4]))
        elif tmp[1]=="p2a": 
            if tmp[3]=='0 fnt':
                line[3][line[1][1]-1][13] = 0.0
            else:
                line[3][line[1][1]-1][13] = float(tmp[3])/float(re.sub('[^0-9]+', '',tmp[4]))
        else: 
            if tmp[3]=='0 fnt':
                line[3][line[1][2]-1][13] = 0.0
            else:
                line[3][line[1][2]-1][13] = float(tmp[3])/float(re.sub('[^0-9]+', '',tmp[4]))
        
    elif batlog.startswith("|switch|"):
        if switchNo>0:
            tmp = re.findall(r"[\w'“”(),&.’ -]+",(batlog))
            CountSw = switchNo-1
            if tmp[1]=="p1a": 
                placeholder = line[0][0]
                line[0][0] = line[0][-line[10][0]-1]
                line[0][-line[10][0]-1] = placeholder
                clearboost(line, 2, 0, -line[10][0]-1)
            elif tmp[1]=="p1b": 
                placeholder = line[0][1]
                line[0][1] = line[0][-line[10][1]-1]
                line[0][-line[10][1]-1] = placeholder
                clearboost(line, 2, 1, -line[10][1]-1)
            elif tmp[1]=="p2a":
                placeholder = line[1][0]
                line[1][0] = line[1][-line[11][0]-1]
                line[1][-line[11][0]-1] = placeholder
                clearboost(line, 3, 0, -line[11][0]-1)
            else: 
                placeholder = line[1][1]
                line[1][1] = line[1][-line[11][1]-1]
                line[1][-line[11][1]-1] = placeholder
                clearboost(line, 3, 1, -line[11][1]-1)
        else:
            t, _, double_switch =readaction(inputlog,k)

            try:
                t.remove(None)
            except (RuntimeError, IndexError, ValueError):
                None
            tmp = re.findall(r"[\w'“”(),&.’ -]+",(batlog))
            if tmp[1]=="p1a": 
                placeholder = line[0][0]
                line[0][0] = line[0][-t[0]-1]
                line[0][-t[0]-1] = placeholder
                clearboost(line, 2, 0, -t[0]-1)
            elif tmp[1]=="p1b": 
                placeholder = line[0][1]
                line[0][1] = line[0][-t[len(t)-1]-1]
                line[0][-t[len(t)-1]-1] = placeholder
                clearboost(line, 2, 1, -t[len(t)-1]-1)

            elif tmp[1]=="p2a":
                placeholder = line[1][0]
                line[1][0] = line[1][-t[0]-1]
                line[1][-t[0]-1] = placeholder
                clearboost(line, 3, 0, -t[0]-1)
            else: 
                placeholder = line[1][1]
                line[1][1] = line[1][-t[len(t)-1]-1]
                line[1][-t[len(t)-1]-1] = placeholder
                clearboost(line, 3, 1, -t[len(t)-1]-1)

            k = k+1-(double_switch - DD) # double killed switching adjustment
            double_switch = max(double_switch-DD,0)
            
    elif batlog.startswith("|swap|"):
        None
        
    elif batlog.startswith("|-weather|"):
        tmp = re.findall(r"[\w'“”(),&.’ -]+",(batlog)) #(wea, weather, ...)
        wea = weather[tmp[1].upper()]
        if line[4][0]!= wea:
            line[4] = [wea,0]
        
    elif batlog.startswith("|-heal|"):
        tmp = re.findall(r"[\w'“”(),&.’ -]+",(batlog)) # (dam,pos,Pok,hp-new,hp-old)
        if tmp[1]=="p1a": 
            line[2][line[0][1]-1][13] = float(tmp[3])/float(re.sub('[^0-9]+', '',tmp[4]))
        elif tmp[1]=="p1b": 
            line[2][line[0][2]-1][13] = float(tmp[3])/float(re.sub('[^0-9]+', '',tmp[4]))
        elif tmp[1]=="p2a": 
            line[3][line[1][1]-1][13] = float(tmp[3])/float(re.sub('[^0-9]+', '',tmp[4]))
        else: 
            line[3][line[1][2]-1][13] = float(tmp[3])/float(re.sub('[^0-9]+', '',tmp[4]))
            
    elif batlog.startswith("|-enditem|"):
        tmp = re.findall(r"[\w'“”(),&.’ -]+",(batlog))
        if tmp[1]=="p1a": 
            line[2][line[0][1]-1][1] = 0
        elif tmp[1]=="p1b": 
            line[2][line[0][2]-1][1] = 0
        elif tmp[1]=="p2a": 
            line[3][line[1][1]-1][1] = 0
        else: 
            line[3][line[1][2]-1][1] = 0
        
    elif batlog.lower().startswith("|-fieldstart|move: tri"):
        line[7] = [1,0]
    elif batlog.lower().startswith("|-fieldend|move: tri"):
        line[7] = [0,0]
        
    elif batlog.lower().startswith("|-fieldstart|move: grav"):
        tmp = re.findall(r"[\w'“”(),&.’ -]+",(batlog))
        line[6] = [extra_field[tmp[2].upper()],0]
    elif batlog.lower().startswith("|-fieldend|move: grav"):
        tmp = re.findall(r"[\w'“”(),&.’ -]+",(batlog))
        line[6] = [0,0]
        
    elif batlog.startswith("|-fieldstart|"):
        tmp = re.findall(r"[\w'“”(),&.’ -]+",(batlog))
        line[5] = [terrian[tmp[2].upper()],0]
    elif batlog.startswith("|-fieldend|"):
        line[5] = [0,0]
    
    elif batlog.startswith("|-status|"):
        tmp = re.findall(r"[\w'“”(),&.’ -]+",(batlog))
        if tmp[1]=="p1a": 
            line[2][line[0][1]-1][14] = sp_status[tmp[3].upper()]
            line[2][line[0][1]-1][15] = 0
        elif tmp[1]=="p1b": 
            line[2][line[0][2]-1][14] = sp_status[tmp[3].upper()]
            line[2][line[0][2]-1][15] = 0
        elif tmp[1]=="p2a": 
            line[3][line[1][1]-1][14] = sp_status[tmp[3].upper()]
            line[3][line[1][1]-1][15] = 0
        else:
            line[3][line[1][2]-1][14] = sp_status[tmp[3].upper()]
            line[3][line[1][2]-1][15] = 0
        
    elif batlog.startswith("|-curestatus|"):
        tmp = re.findall(r"[\w'“”(),&.’ -]+",(batlog))
        if tmp[1]=="p1a": 
            line[2][line[0][1]-1][14] = 0
            line[2][line[0][1]-1][15] = 0
        elif tmp[1]=="p1b": 
            line[2][line[0][2]-1][14] = 0
            line[2][line[0][2]-1][15] = 0
        elif tmp[1]=="p2a": 
            line[3][line[1][1]-1][14] = 0
            line[3][line[1][1]-1][15] = 0
        else: 
            line[3][line[1][2]-1][14] = 0
            line[3][line[1][2]-1][15] = 0
        
    elif batlog.startswith("|-sidesta"):
        pos = int(re.sub('[^0-9]+', '',(batlog))==p2)+8 #(8 for team1, 9 team2)
        tmp = side_con[re.findall(r"[^(.|)]+",(batlog))[1].upper()] #(indicator, effect)
        line[pos][tmp] = 1
            
    elif batlog.startswith("|-sideend"):
        pos = int(re.sub('[^0-9]+', '',(batlog))==p2)+8 #(8 for team1, 9 team2)
        tmp = side_con[re.findall(r"[^(.|)]+",(batlog))[1].upper()] #(indicator, effect)
        line[pos][tmp] = 0
            
    elif batlog.startswith("|-start|"):  #yawn special
        tmp = re.findall(r"[\w'“”(),&.’ -]+",(batlog))
        try:
            if tmp[4].lower()=='yawn':
                if tmp[1]=="p1a": 
                    line[2][line[0][1]-1][14] = 2
                elif tmp[1]=="p1b": 
                    line[2][line[0][2]-1][14] = 2
                elif tmp[1]=="p2a": 
                    line[3][line[1][1]-1][14] = 2
                else:
                    line[3][line[1][2]-1][14] = 2
        except (RuntimeError, IndexError, NameError):
            None

    elif batlog.startswith("|win|"):
        None
    #'|-sethp| 
        
    return current, upflag, k, CountSw, double_switch

In [ ]:
def readlog (inputlog, batlog, team1, team2, win, p2): # save features and outcome in the beginning of the turn
    
    record = [] 
    
    k = 6 # line of inputlog
    CountSw = 0
    pos1 = [int(inputlog[4][9]),int(inputlog[4][12]),int(inputlog[4][15]),int(inputlog[4][18])] # team 1 position
    pos2 = [int(inputlog[5][9]),int(inputlog[5][12]),int(inputlog[5][15]),int(inputlog[5][18])] # team 2 position
    DD = 0
    
    line = [None]*13 # status of enviornment
    line[0] = pos1 # team 1 position
    line[1] = pos2 # team 2 position
    line[2] = team1 # team 1
    line[3] = team2 # team 1
    line[4] = [0,0] # weather
    line[5] = [0,0] # Terrian
    line[6] = [0,0] # extra_field
    line[7] = [0,0] # trickroom
    line[8] = [0]*19 # team1 sidecon
    line[9] = [0]*19 # team2 sidecon
    line[10] = [0]*2  # team1 action
    line[11] = [0]*2  # team1 action
    line[12] = [win]
    
    record.append(copy.deepcopy(line))
    
    for i in range(35, len(batlog)):
        #print (i, k, CountSw, DD)
        line, flag, k, CountSw, DD  = readbatlog(line, batlog[i-1], inputlog, k, CountSw, p2, DD)
        if flag==1:
            record.append(copy.deepcopy(line))
        
    return record

In [ ]:
import os          # process all log
from keras.utils import to_categorical

directory_in_str = r"C:\Users\Cyril\Documents\project\Folder1"
directory = os.fsencode(directory_in_str)
    
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".json"): 
        with open(directory_in_str +'\\' + filename,"rb") as log_file:
            log = json.load(log_file)
            print(filename)
        
        try:
            team1, team2 = read_teams(log['p1team'], log['p2team'])
            win = log['winner']==log['p1']
            tmp = readlog(log['inputLog'], log['log'], team1, team2, int(win), log['p2'])
            
        
            for i in range(0,len(tmp)):
                tmp[i][2]=sum(tmp[i][2],[])
                tmp[i][3]=sum(tmp[i][3],[])
                tmp[i]=sum(tmp[i],[])
        
        except (Exception) as e:
            print(e)
            continue

        
        df.to_csv(r'C:\Users\Cyril\Documents\project\out\\' + filename +'.csv',index=False,header=False)